In [ ]:
!pip install sshtunnel cassandra-driver

#https://medium.com/@amirziai/query-your-database-over-an-ssh-tunnel-with-pandas-603ce49b35a1
#https://medium.com/@upu1994/connect-to-cassandra-with-python-3-x-and-get-pandas-dataframe-81adbf1a09a9
#https://blog.ruanbekker.com/blog/2018/04/23/setup-a-ssh-tunnel-with-the-sshtunnel-module-in-python/

In [ ]:
from sshtunnel import SSHTunnelForwarder

#ssh config
host = '********.sa-east-1.compute.amazonaws.com'
ssh_username = 'ubuntu'
ssh_private_key = 'cassandra.pem'
local_host = '127.0.0.1'
local_port = 9042

server = SSHTunnelForwarder(
          (host, 22),
          ssh_username=ssh_username,
          ssh_private_key=ssh_private_key,
          remote_bind_address=(local_host, local_port),
          local_bind_address=(local_host, local_port))
server.start()
server

<class 'sshtunnel.SSHTunnelForwarder'> object
ssh gateway: ec2-18-230-45-203.sa-east-1.compute.amazonaws.com:22
proxy: no
username: ubuntu
authentication: {'pkeys': [('ssh-rsa', b'71fc0486a53912f3d38d7ac484ba485e')]}
hostkey: not checked
status: started
keepalive messages: every 5.0 sec
tunnel connection check: disabled
concurrent connections: allowed
compression: not requested
logging level: ERROR
local binds: [('127.0.0.1', 9042)]
remote binds: [('127.0.0.1', 9042)]

In [ ]:
from cassandra.cluster import Cluster 
from cassandra.auth import PlainTextAuthProvider 
import pandas as pd 

class ConnDao(object):
      def __init__(self):
         self.cluster = None
         self.username = 'admin'
         self.password = ''
         self.host = '127.0.0.1'
         self.keyspace = 'keyspace_name'
         self.session = self.createSession()

      def __del__(self):
         self.cluster.shutdown()

      def createSession(self):
         self.cluster = Cluster([self.host],                                            auth_provider=PlainTextAuthProvider(username=self.username, password=self.password))         
         self.session = self.cluster.connect(self.keyspace)         
         return self.session       
      
      def getSession(self):
         return self.session

      def get_data(self):
         query = """SELECT * from  %s.teste""" %self.keyspace         
         data = pd.DataFrame(self.getSession().execute(query, timeout=None))         
         return data

In [ ]:
conexao = ConnDao()
data = conexao.get_data()

In [ ]:
data.head()